<a href="https://colab.research.google.com/github/hjdeck/Cyberbullying-Classification/blob/main/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import nltk
from nltk import word_tokenize
nltk.download('punkt')

df = pd.read_csv('cyberbullying_tweets.csv') 

data = df['tweet_text'].map(word_tokenize).values
total_vocabulary = set(word.lower() for tweet in data for word in tweet) 
print('There are {} unique words in the dataset.'.format(len(total_vocabulary)))
print('There are {} unique tweets in the dataset.'.format(len(data)))

df.head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


There are 65914 unique words in the dataset.
There are 47692 unique tweets in the dataset.


,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying


In [3]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dense, LSTM, Embedding
from keras.layers import Dropout, Activation, Bidirectional, GlobalMaxPool1D
from keras.models import Sequential
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.preprocessing import text, sequence

# set the emotion/sentiment as our target
target = df['cyberbullying_type']

# use one hot encoding since our target is categorical
y = pd.get_dummies(target).values

# use keras to create a Tokenizer object
tokenizer = text.Tokenizer(num_words=20000)  # limit to the num_words most important ones
tokenizer.fit_on_texts(list(df['tweet_text']))
tokenized_texts = tokenizer.texts_to_sequences(df['tweet_text'])
X = pad_sequences(tokenized_texts, maxlen=100)

In [4]:

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# create my NN model
model = Sequential()

embedding_size = 128
model.add(Embedding(len(total_vocabulary), embedding_size))
model.add(LSTM(25, return_sequences=True))
model.add(GlobalMaxPool1D())
model.add(Dropout(0.5))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(6, activation='softmax'))  # use 4 because we have 4 categories

In [5]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
model.summary() # check the shape

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 128)         8436992   
                                                                 
 lstm (LSTM)                 (None, None, 25)          15400     
                                                                 
 global_max_pooling1d (Globa  (None, 25)               0         
 lMaxPooling1D)                                                  
                                                                 
 dropout (Dropout)           (None, 25)                0         
                                                                 
 dense (Dense)               (None, 50)                1300      
                                                                 
 dropout_1 (Dropout)         (None, 50)                0         
                                                        

In [6]:
model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.1)

Epoch 1/10
537/537 [==============================] - 162s 295ms/step - loss: 1.0015 - accuracy: 0.5870 - val_loss: 0.5186 - val_accuracy: 0.7529
Epoch 2/10
537/537 [==============================] - 144s 268ms/step - loss: 0.5655 - accuracy: 0.7623 - val_loss: 0.4765 - val_accuracy: 0.7972
Epoch 3/10
537/537 [==============================] - 176s 328ms/step - loss: 0.4857 - accuracy: 0.7864 - val_loss: 0.4748 - val_accuracy: 0.7943
Epoch 4/10
537/537 [==============================] - 154s 286ms/step - loss: 0.4375 - accuracy: 0.8082 - val_loss: 0.4659 - val_accuracy: 0.8179
Epoch 5/10
537/537 [==============================] - 160s 298ms/step - loss: 0.4206 - accuracy: 0.8204 - val_loss: 0.4654 - val_accuracy: 0.8242
Epoch 6/10
537/537 [==============================] - 152s 283ms/step - loss: 0.3753 - accuracy: 0.8442 - val_loss: 0.5102 - val_accuracy: 0.8205
Epoch 7/10
537/537 [==============================] - 147s 274ms/step - loss: 0.3592 - accuracy: 0.8587 - val_loss: 0.4829 -

In [8]:
model.save('RNN_model')